In [ ]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
def reproject_images(input_folder, output_folder, target_crs='EPSG:4326'):
    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}.tif')

        with rasterio.open(input_path) as src:
            transform, width, height = calculate_default_transform(src.crs, target_crs, src.width, src.height, *src.bounds)

            dst_meta = src.meta.copy()
            dst_meta.update({'crs': target_crs, 'transform': transform, 'width': width, 'height': height})

            with rasterio.open(output_path, 'w', **dst_meta) as dst:
                for i in range(1, src.count + 1):
                    band = src.read(i)
                    reproject(
                        source=band,
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=target_crs,
                        resampling=Resampling.nearest
                    )

# Example Usage:
input_folder_path = '/path/to/input/folder'
output_folder_path = '/path/to/output/folder'

reproject_images(input_folder_path, output_folder_path)